## Practice Activity 7.2 - Chapter 13 

Consider four possible models for predicting house prices:

1. Using only the size and number of rooms.
2. Using size, number of rooms, and building type.
3. Using size and building type, and their interaction.
4. Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.

Set up a pipeline for each of these four models.

Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

Note: You should only use the function train_test_split() one time in your code; that is, we should be predicting on the same test set for all three models.

In [83]:
import pandas as pd
from palmerpenguins import load_penguins
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np


data = pd.read_csv("C:/Users/alexa/OneDrive/Documentos/VSCode Folder/GSB544_Computing_and_ML/Data/In Class Data/AmesHousing.csv")
data.head()

,2Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [179]:
# Model 1 - size and # of rooms
X = data.drop("SalePrice", axis = 1)
y = data[["SalePrice"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LinearRegression()
enc = OneHotEncoder()
ss = StandardScaler()

ct = ColumnTransformer(
    [('standardize', ss, ['Gr Liv Area', 'TotRms AbvGrd'])], remainder='drop'
).set_output(transform="pandas")

ct.fit_transform(X_train)

pipeline = Pipeline([
    ('transform', ct),
    ('ols', lr)]
)

pipeline_fitted = pipeline.fit(X_train, y_train)

y_pred = pipeline_fitted.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R-squared:", r2)

RMSE: 61928.53719680032
R-squared: 0.5216562779062477


In [180]:
# Model 2 - size, # of rooms, and building type
ct = ColumnTransformer(
    [('standardize', ss, ['Gr Liv Area', 'TotRms AbvGrd']),
     ('dummify', OneHotEncoder(drop='first', sparse_output=False), ['Bldg Type'])], remainder="drop"
).set_output(transform="pandas")

ct.fit_transform(X_train)

pipeline = Pipeline([
    ('transform', ct),
    ('ols', lr)]
)

pipeline_fitted = pipeline.fit(X_train, y_train)

y_pred = pipeline_fitted.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)


print("RMSE:", rmse)
print("R-squared:", r2)

RMSE: 59589.20317423357
R-squared: 0.5571123284150978


In [181]:
# Model 3 - size, building type, and interactions
ct_pre = ColumnTransformer(
    [
        ('standardize', StandardScaler(), ['Gr Liv Area']),
        ('encode', OneHotEncoder(drop='first', sparse_output=False), ['Bldg Type'])
    ],
    remainder="drop"
).set_output(transform="pandas")

ct_inter = ColumnTransformer(
    [
        ('interaction1', PolynomialFeatures(interaction_only=True, include_bias=False), ['standardize__Gr Liv Area', 'encode__Bldg Type_2fmCon']),
        ('interaction2', PolynomialFeatures(interaction_only=True, include_bias=False), ['standardize__Gr Liv Area', 'encode__Bldg Type_Duplex']),
        ('interaction3', PolynomialFeatures(interaction_only=True, include_bias=False), ['standardize__Gr Liv Area', 'encode__Bldg Type_Twnhs']),
        ('interaction4', PolynomialFeatures(interaction_only=True, include_bias=False), ['standardize__Gr Liv Area', 'encode__Bldg Type_TwnhsE'])
    ]
    ,remainder="drop"
).set_output(transform="pandas")


# ct_pre.fit_transform(X_train)

ct_inter.fit_transform(ct_pre.fit_transform(X_train))

# # Define pipeline
pipeline = Pipeline([
    ('preprocessing', ct_pre),
    ('interaction', ct_inter),
    ('ols', LinearRegression())
])

# Fit pipeline
pipeline_fitted = pipeline.fit(X_train, y_train)


# # Predict and evaluate
y_pred = pipeline_fitted.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)


pipeline.named_steps['interaction'].get_feature_names_out()

print("RMSE:", rmse)
print("R-squared:", r2)


RMSE: 58349.8651800962
R-squared: 0.5753431353481266


In [182]:
# Model 4 - 5-degree polynomial on size, a 5-degree polynomial on # of rooms, and also building type

ct_pre = ColumnTransformer(
    [
        ('encode', OneHotEncoder(sparse_output=False), ['Bldg Type']),
        ('poly_size', Pipeline([
           ("standardize", StandardScaler()),
           ("poly", PolynomialFeatures(degree = 5, include_bias=False))
        ]), ['Gr Liv Area']),
        ('poly_rooms', Pipeline([
            ("standardize", StandardScaler()),
            ("poly", PolynomialFeatures(degree = 5, include_bias=False))    
        ]), ["TotRms AbvGrd"])
    ],
    remainder="drop"
).set_output(transform="pandas")


ct_pre.fit_transform(X_train)

# ct_poly.fit_transform(ct_pre.fit_transform(X_train))

pipeline = Pipeline([
    ('preprocessing', ct_pre),
    ('ols', LinearRegression())
])

# Fit pipeline
pipeline_fitted = pipeline.fit(X_train, y_train)

# # Predict and evaluate
y_pred = pipeline_fitted.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)


print("RMSE:", rmse)
print("R-squared:", r2)

RMSE: 61742.47507941377
R-squared: 0.5245262938514901


### Model 3 with size, building type, and interactions performed best


Once again consider four modeling options for house price:

1. Using only the size and number of rooms.
2. Using size, number of rooms, and building type.
3. Using size and building type, and their interaction.
4. Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.

Use cross_val_score with the pipelines you made earlier to find the cross-validated root mean squared error for each model.

Which do you prefer? Does this agree with your conclusion from earlier?

In [ ]:
# Model 1 using cross_val_score
from sklearn.model_selection import cross_val_score

lr = LinearRegression()
enc = OneHotEncoder()
ss = StandardScaler()

ct = ColumnTransformer(
    [('standardize', ss, ['Gr Liv Area', 'TotRms AbvGrd'])], remainder='drop'
).set_output(transform="pandas")

ct.fit_transform(X_train)

mod_1_pipeline = Pipeline([
    ('transform', ct),
    ('ols', lr)]
)

scores = cross_val_score(mod_1_pipeline, X, y, cv=5, scoring='neg_root_mean_squared_error')
scores = abs(scores)
mean_score = scores.mean()

R_squared_scores = cross_val_score(mod_1_pipeline, X, y, cv=5, scoring='r2')
mean_R2 = R_squared_scores.mean()

print("Cross val RMSE =", mean_score)
print("Cross val R-squared:", mean_R2)

Cross val RMSE = 55806.32634926364
Cross val R-squared: 0.504208752508862


In [194]:
# Model 2 using cross_val_score
ct = ColumnTransformer(
    [('standardize', ss, ['Gr Liv Area', 'TotRms AbvGrd']),
     ('dummify', OneHotEncoder(drop='first', sparse_output=False), ['Bldg Type'])], remainder="drop"
).set_output(transform="pandas")

ct.fit_transform(X_train)

mod_2_pipeline = Pipeline([
    ('transform', ct),
    ('ols', lr)]
)

scores = cross_val_score(mod_2_pipeline, X, y, cv=5, scoring='neg_root_mean_squared_error')
scores = abs(scores)
mean_score = scores.mean()
R_squared_scores = cross_val_score(mod_2_pipeline, X, y, cv=5, scoring='r2')
mean_R2 = R_squared_scores.mean()

print("Cross val RMSE =", mean_score)
print("Cross val R-squared:", mean_R2)

Cross val RMSE = 54168.081429193844
Cross val R-squared: 0.5328824390692034


In [195]:
# Model 3 using cross_val_score
ct_pre = ColumnTransformer(
    [
        ('standardize', StandardScaler(), ['Gr Liv Area']),
        ('encode', OneHotEncoder(drop='first', sparse_output=False), ['Bldg Type'])
    ],
    remainder="drop"
).set_output(transform="pandas")

ct_inter = ColumnTransformer(
    [
        ('interaction1', PolynomialFeatures(interaction_only=True, include_bias=False), ['standardize__Gr Liv Area', 'encode__Bldg Type_2fmCon']),
        ('interaction2', PolynomialFeatures(interaction_only=True, include_bias=False), ['standardize__Gr Liv Area', 'encode__Bldg Type_Duplex']),
        ('interaction3', PolynomialFeatures(interaction_only=True, include_bias=False), ['standardize__Gr Liv Area', 'encode__Bldg Type_Twnhs']),
        ('interaction4', PolynomialFeatures(interaction_only=True, include_bias=False), ['standardize__Gr Liv Area', 'encode__Bldg Type_TwnhsE'])
    ]
    ,remainder="drop"
).set_output(transform="pandas")

# ct_pre.fit_transform(X_train)

ct_inter.fit_transform(ct_pre.fit_transform(X_train))

# Define pipeline
mod_3_pipeline = Pipeline([
    ('preprocessing', ct_pre),
    ('interaction', ct_inter),
    ('ols', LinearRegression())
])

scores = cross_val_score(mod_3_pipeline, X, y, cv=5, scoring='neg_root_mean_squared_error')
scores = abs(scores)
mean_score = scores.mean()

R_squared_scores = cross_val_score(mod_3_pipeline, X, y, cv=5, scoring='r2')
mean_R2 = R_squared_scores.mean()

print("Cross val RMSE =", mean_score)
print("Cross val R-squared:", mean_R2)

Cross val RMSE = 53429.5538692013
Cross val R-squared: 0.5448852389554741


In [196]:
# Model 4 using cross_val_score
ct_pre = ColumnTransformer(
    [
        ('encode', OneHotEncoder(sparse_output=False), ['Bldg Type']),
        ('poly_size', Pipeline([
           ("standardize", StandardScaler()),
           ("poly", PolynomialFeatures(degree = 5, include_bias=False))
        ]), ['Gr Liv Area']),
        ('poly_rooms', Pipeline([
            ("standardize", StandardScaler()),
            ("poly", PolynomialFeatures(degree = 5, include_bias=False))    
        ]), ["TotRms AbvGrd"])
    ],
    remainder="drop"
).set_output(transform="pandas")


ct_pre.fit_transform(X_train)

mod_4_pipeline = Pipeline([
    ('preprocessing', ct_pre),
    ('ols', LinearRegression())
])

scores = cross_val_score(mod_4_pipeline, X, y, cv=5, scoring='neg_root_mean_squared_error')
scores = abs(scores)
mean_score = scores.mean()

R_squared_scores = cross_val_score(mod_4_pipeline, X, y, cv=5, scoring='r2')
mean_R2 = R_squared_scores.mean()

print("Cross val RMSE =", mean_score)
print("Cross val R-squared:", mean_R2)


Cross val RMSE = 55176.96594338035
Cross val R-squared: 0.5106643234404193


### I prefer the cross validation method as it seems more efficient while providing all parts of the data to be both test and training data. Each RMSE value is lower and model 3 is still the best.

Consider one hundred modeling options for house price:

- House size, trying degrees 1 through 10
- Number of rooms, trying degrees 1 through 10
- Building Type

Hint: The dictionary of possible values that you make to give to GridSearchCV will have two elements instead of one.

Q1: Which model performed the best?

Q2: What downsides do you see of trying all possible model options? How might you go about choosing a smaller number of tuning values to try?

In [117]:
# Model 1 house size with degrees 1 through 10
from sklearn.model_selection import GridSearchCV

ct_poly = ColumnTransformer(
  [
    ("standardize", StandardScaler(), ["Gr Liv Area"]),
    ("polynomial", PolynomialFeatures(), ["Gr Liv Area"])
  ],
  remainder = "drop"
)

lr_pipeline_poly = Pipeline(
  [("preprocessing", ct_poly),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")

degrees = {'preprocessing__polynomial__degree': np.arange(1, 11)}

gscv = GridSearchCV(lr_pipeline_poly, degrees, cv = 5, scoring='r2')


gscv_fitted = gscv.fit(X, y)

gscv_fitted.cv_results_

gscv_fitted.cv_results_['mean_test_score']

pd.DataFrame(data = {"degrees": np.arange(1, 11), "scores": gscv_fitted.cv_results_['mean_test_score']})

,degrees,scores
0,1,0.488503
1,2,0.490241
2,3,0.507396
3,4,0.499218
4,5,0.451860
5,6,0.333837
6,7,0.029322
7,8,-0.968096
8,9,-4.545599
9,10,-16.187997


In [118]:
# Model 2 # of rooms with degrees 1 through 10

ct_poly = ColumnTransformer(
  [
    ("standardize", StandardScaler(), ["TotRms AbvGrd"]),
    ("polynomial", PolynomialFeatures(), ["TotRms AbvGrd"])
  ],
  remainder = "drop"
)

lr_pipeline_poly = Pipeline(
  [("preprocessing", ct_poly),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")

degrees = {'preprocessing__polynomial__degree': np.arange(1, 11)}

gscv = GridSearchCV(lr_pipeline_poly, degrees, cv = 5, scoring='r2')


gscv_fitted = gscv.fit(X, y)

gscv_fitted.cv_results_

gscv_fitted.cv_results_['mean_test_score']

pd.DataFrame(data = {"degrees": np.arange(1, 11), "scores": gscv_fitted.cv_results_['mean_test_score']})

,degrees,scores
0,1,0.231039
1,2,0.226852
2,3,0.235220
3,4,0.233668
4,5,0.221095
5,6,0.136030
6,7,0.203600
7,8,-0.431101
8,9,-0.318079
9,10,-630.898354


In [119]:
# Model 3 building type

ct = ColumnTransformer(
    [('dummify', OneHotEncoder(drop='first', sparse_output=False), ['Bldg Type'])], remainder="drop"
).set_output(transform="pandas")

ct.fit_transform(X_train)

mod_2_pipeline = Pipeline([
    ('transform', ct),
    ('ols', lr)]
)

scores = cross_val_score(mod_2_pipeline, X, y, cv=5, scoring='r2')
scores = abs(scores)
mean_score = scores.mean()

print("Cross val R-squared =", mean_score)

Cross val R-squared = 0.024446335977577283


Q1: Which model performed the best?

Model 1 (house size) with degree 3 performed the best.

Q2: What downsides do you see of trying all possible model options? How might you go about choosing a smaller number of tuning values to try?

Downsides of tuning might be that it takes up way too much computing power and is innefficient in the sense that it computes more scores that are unneccesary compared with scores that are necessary. You could just try a smaller range of tuning values two or three times to disperse the computing power, or you could do a few individual polynomial models of very different degrees to find a range that would be more likely to capture the best model. 